In [1]:
import pandas as pd

df=pd.read_csv("csvs/2019-04-01.csv")

In [2]:
for i in range(2,8):
    print("csvs/2019-04-{}.csv".format('0'+str(i)))
    temp=pd.read_csv("csvs/2019-04-{}.csv".format('0'+str(i)), low_memory=False)
    df=pd.concat([df,temp])

csvs/2019-04-02.csv
csvs/2019-04-03.csv
csvs/2019-04-04.csv
csvs/2019-04-05.csv
csvs/2019-04-06.csv
csvs/2019-04-07.csv


In [3]:
df.inventory_source.unique()

array(['MOPUB', 'GOOGLE_ADX'], dtype=object)

In [4]:
df.creative_type.unique()

array(['video', 'banner', nan], dtype=object)

In [5]:
X=df[['inventory_source','creative_type','bid_floor']].values
X

array([['MOPUB', 'video', 0.00337],
       ['MOPUB', 'video', 0.00337],
       ['MOPUB', 'video', 0.00337],
       ...,
       ['MOPUB', nan, 0.0020800000000000003],
       ['MOPUB', nan, 4e-05],
       ['MOPUB', nan, 0.00085]], dtype=object)

In [6]:
y=df['clicks'].values
y

array([0., 0., 0., ..., 0., 0., 0.])

In [7]:
for i in range(len(X)):
    X[i][0]= int(X[i][0]=='MOPUB')
    if X[i][1]=='video':
        X[i][1]=0
    elif X[i][1]=='banner':
        X[i][1]=1
    else:
        X[i][1]=-1
        
X

array([[1, 0, 0.00337],
       [1, 0, 0.00337],
       [1, 0, 0.00337],
       ...,
       [1, -1, 0.0020800000000000003],
       [1, -1, 4e-05],
       [1, -1, 0.00085]], dtype=object)

In [8]:
from imblearn.under_sampling import RandomUnderSampler

X_resampled, y_resampled = RandomUnderSampler(sampling_strategy='majority',random_state=100).fit_resample(X, y)

In [9]:
X_resampled

array([[1, -1, 7e-05],
       [1, -1, 8.999999999999999e-05],
       [1, -1, 0.00139],
       ...,
       [1, -1, 0.0020800000000000003],
       [1, -1, 0.0017699999999999999],
       [1, -1, 0.00385]], dtype=object)

In [10]:
y_resampled

array([0., 0., 0., ..., 1., 1., 1.])

In [11]:
len(X_resampled), len(X)

(285860, 5021077)

In [12]:
len(y_resampled), len(y)

(285860, 5021077)

In [30]:
import pandas as pd

test=pd.read_csv("csvs/2019-04-08.csv", low_memory=False)

In [14]:
test.inventory_source.unique()

array(['MOPUB'], dtype=object)

In [15]:
test.creative_type.unique()

array(['video', 'banner', nan], dtype=object)

In [16]:
X_test=test[['inventory_source','creative_type','bid_floor']].values
X_test

array([['MOPUB', 'video', 0.00099],
       ['MOPUB', 'video', 0.00505],
       ['MOPUB', 'video', 0.0016899999999999999],
       ...,
       ['MOPUB', nan, 0.0016699999999999998],
       ['MOPUB', nan, 0.005889999999999999],
       ['MOPUB', nan, 0.00673]], dtype=object)

In [17]:
y_test=test['installs'].values
y_test

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
len(X_test), len(y_test)

(1350203, 1350203)

In [19]:
for i in range(len(X_test)):
    X_test[i][0]= int(X_test[i][0]=='MOPUB')
    if X_test[i][1]=='video':
        X_test[i][1]=0
    elif X_test[i][1]=='banner':
        X_test[i][1]=1
    else:
        X_test[i][1]=-1
        
X_test

array([[1, 0, 0.00099],
       [1, 0, 0.00505],
       [1, 0, 0.0016899999999999999],
       ...,
       [1, -1, 0.0016699999999999998],
       [1, -1, 0.005889999999999999],
       [1, -1, 0.00673]], dtype=object)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, balanced_accuracy_score

LR=LogisticRegression()
LR.fit(X_resampled,y_resampled)
y_pred = LR.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

         0.0       1.00      0.47      0.64   1350126
         1.0       0.00      0.75      0.00        77

   micro avg       0.47      0.47      0.47   1350203
   macro avg       0.50      0.61      0.32   1350203
weighted avg       1.00      0.47      0.64   1350203



(0.46707717283993594, 0.610153802672501)

In [27]:
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier()

bc.fit(X_resampled, y_resampled) 
y_pred = bc.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75   1350126
         1.0       0.00      0.60      0.00        77

   micro avg       0.60      0.60      0.60   1350203
   macro avg       0.50      0.60      0.38   1350203
weighted avg       1.00      0.60      0.75   1350203



(0.6010051821837161, 0.5992039925239493)

In [28]:
from sklearn import ensemble

GBclf = ensemble.GradientBoostingClassifier()
GBclf.fit(X_resampled,y_resampled)
y_predict=GBclf.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75   1350126
         1.0       0.00      0.60      0.00        77

   micro avg       0.60      0.60      0.60   1350203
   macro avg       0.50      0.60      0.38   1350203
weighted avg       1.00      0.60      0.75   1350203



(0.6010051821837161, 0.5992039925239493)

In [29]:
from sklearn.ensemble import AdaBoostClassifier

ABclf=AdaBoostClassifier()
ABclf.fit(X_resampled, y_resampled)
y_predict=ABclf.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75   1350126
         1.0       0.00      0.60      0.00        77

   micro avg       0.60      0.60      0.60   1350203
   macro avg       0.50      0.60      0.38   1350203
weighted avg       1.00      0.60      0.75   1350203



(0.6010051821837161, 0.5992039925239493)

In [24]:
from sklearn.ensemble import RandomForestClassifier

RFclf=RandomForestClassifier()
RFclf.fit(X_resampled, y_resampled)
y_predict=RFclf.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75   1350126
         1.0       0.00      0.60      0.00        77

   micro avg       0.60      0.60      0.60   1350203
   macro avg       0.50      0.60      0.37   1350203
weighted avg       1.00      0.60      0.75   1350203



(0.5982907755352344, 0.5978467117960765)

In [32]:
from sklearn.ensemble import VotingClassifier

Vclf=VotingClassifier(
    estimators=[('Bagging', bc), ('GradientBoost', GBclf), ('AdaBoost',ABclf),('RandomForest', RFclf)],
    voting='soft')
Vclf.fit(X_resampled, y_resampled)
y_predict=Vclf.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75   1350126
         1.0       0.00      0.60      0.00        77

   micro avg       0.60      0.60      0.60   1350203
   macro avg       0.50      0.60      0.38   1350203
weighted avg       1.00      0.60      0.75   1350203



(0.6010051821837161, 0.5992039925239493)

In [36]:
from sklearn.neural_network import MLPClassifier

MLPclf = MLPClassifier(solver='lbfgs', alpha=1e-8, activation='logistic', 
                       hidden_layer_sizes=(5, 5), max_iter=1000, random_state=1)

MLPclf.fit(X_resampled, y_resampled)
y_predict=MLPclf.predict(X_test)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred, average='micro'), balanced_accuracy_score(y_test, y_pred)


              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75   1350126
         1.0       0.00      0.60      0.00        77

   micro avg       0.60      0.60      0.60   1350203
   macro avg       0.50      0.60      0.38   1350203
weighted avg       1.00      0.60      0.75   1350203



(0.6010051821837161, 0.5992039925239493)